In [5]:
import tkinter as tk
from tkinter import ttk, messagebox, simpledialog
import pandas as pd
import psycopg2
import webbrowser
from typing import List, Callable
from datetime import datetime
from tkcalendar import DateEntry

class DataChangeManager:
    """Класс для управления уведомлениями об изменениях данных"""
    def __init__(self):
        self._subscribers: List[Callable] = []
    
    def subscribe(self, callback: Callable):
        self._subscribers.append(callback)
    
    def unsubscribe(self, callback: Callable):
        if callback in self._subscribers:
            self._subscribers.remove(callback)
    
    def notify(self):
        for callback in self._subscribers:
            callback()

data_change_manager = DataChangeManager()

DB_CONFIG = {
    'host': 'localhost',
    'port': '5433',
    'database': 'hh_test',
    'user': 'postgres',
    'password': '123'
}

df = pd.DataFrame()
comments = {}

def load_data():
    """Загружает данные из базы данных"""
    global df, comments
    try:
        with psycopg2.connect(**DB_CONFIG) as connect:
            with connect.cursor() as cursor:
                cursor.execute("""
                    SELECT column_name 
                    FROM information_schema.columns 
                    WHERE table_name='info_res' AND column_name='comment'
                """)
                if not cursor.fetchone():
                    cursor.execute("ALTER TABLE info_res ADD COLUMN comment TEXT")
                    connect.commit()
            
            query = '''SELECT info_res.session_id, name_resume, full_url, img_text, 
                      short_url, visit_time, file_creation_time, comment 
                      FROM info_res
                      LEFT JOIN parsing_metadata ON info_res.session_id = parsing_metadata.session_id
                      ORDER BY info_res.id ASC'''
            
            df = pd.read_sql(query, connect)
            df['visit_time'] = pd.to_datetime(df['visit_time'])
            
            comments = {f"I{idx}": row['comment'] for idx, row in df.iterrows() 
                       if pd.notna(row['comment'])}
            
        return True
    except Exception as e:
        messagebox.showerror("Ошибка", f"Не удалось загрузить данные: {e}")
        return False

def save_comment_to_db(row_id, comment):
    """Сохраняет комментарий в базе данных"""
    try:
        with psycopg2.connect(**DB_CONFIG) as connect:
            with connect.cursor() as cursor:
                cursor.execute(
                    "UPDATE info_res SET comment = %s WHERE ctid = "
                    "(SELECT ctid FROM info_res ORDER BY id LIMIT 1 OFFSET %s)",
                    (comment, row_id))
                connect.commit()
        return True
    except Exception as e:
        messagebox.showerror("Ошибка", f"Не удалось сохранить комментарий: {e}")
        return False

def save_comments_to_db(row_ids, comment):
    """Сохраняет комментарии для нескольких записей"""
    try:
        with psycopg2.connect(**DB_CONFIG) as connect:
            with connect.cursor() as cursor:
                for row_id in row_ids:
                    cursor.execute(
                        "UPDATE info_res SET comment = %s WHERE ctid = "
                        "(SELECT ctid FROM info_res ORDER BY id LIMIT 1 OFFSET %s)",
                        (comment, row_id))
                connect.commit()
        return True
    except Exception as e:
        messagebox.showerror("Ошибка", f"Не удалось сохранить комментарии: {e}")
        return False

class CommentEditor:
    """Окно редактирования комментария"""
    def __init__(self, parent, item, current_value):
        self.editor = tk.Toplevel(parent)
        self.editor.title("Редактирование комментария")
        self.editor.geometry("400x300")
        self.editor.grab_set()
        
        self.parent = parent
        self.item = item
        self.row_id = int(item[1:])
        
        main_frame = ttk.Frame(self.editor)
        main_frame.pack(fill=tk.BOTH, expand=True, padx=10, pady=10)
        
        self.text_area = tk.Text(main_frame, wrap=tk.WORD, height=10)
        self.text_area.insert("1.0", current_value)
        self.text_area.pack(fill=tk.BOTH, expand=True)
        
        button_frame = ttk.Frame(main_frame)
        button_frame.pack(fill=tk.X, pady=(10, 0))
        
        ttk.Button(button_frame, text="Сохранить", command=self.save_changes).pack(side=tk.LEFT, padx=5)
        ttk.Button(button_frame, text="Очистить", command=self.clear_comment).pack(side=tk.LEFT, padx=5)
        ttk.Button(button_frame, text="Отмена", command=self.editor.destroy).pack(side=tk.RIGHT, padx=5)
        
        self.text_area.focus_set()
    
    def save_changes(self):
        """Сохраняет изменения комментария"""
        new_comment = self.text_area.get("1.0", "end-1c")
        if save_comment_to_db(self.row_id, new_comment):
            df.at[self.row_id, 'comment'] = new_comment
            if new_comment:
                comments[self.item] = new_comment
            elif self.item in comments:
                del comments[self.item]
            data_change_manager.notify()
            self.editor.destroy()
    
    def clear_comment(self):
        """Очищает комментарий"""
        if save_comment_to_db(self.row_id, ""):
            df.at[self.row_id, 'comment'] = ""
            if self.item in comments:
                del comments[self.item]
            data_change_manager.notify()
            self.editor.destroy()
    
class ApplicationWindow:
    """Главное окно приложения"""
    def __init__(self, root):
        self.root = root
        self.root.title("Фильтр резюме HH с комментариями")
        self.root.geometry("1000x700")
        
        self.load_initial_data()
        self.setup_ui()
        data_change_manager.subscribe(self.on_data_changed)
        self.root.protocol("WM_DELETE_WINDOW", self.on_close)
        
        self.refresh_table()
    
    def on_close(self):
        """Обработчик закрытия окна"""
        data_change_manager.unsubscribe(self.on_data_changed)
        self.root.destroy()
    
    def on_data_changed(self):
        """Обработчик изменения данных"""
        self.load_initial_data()
        self.refresh_table()
    
    def setup_ui(self):
        """Настройка пользовательского интерфейса"""
        main_container = ttk.Frame(self.root)
        main_container.pack(fill=tk.BOTH, expand=True)


        self.filter_frame = ttk.Frame(main_container, padding="10")
        self.filter_frame.pack(fill=tk.X)

        # Элементы фильтрации
        ttk.Label(self.filter_frame, text="Ключевое слово в URL:").grid(row=0, column=0, padx=5, pady=5)
        self.keyword_entry = ttk.Entry(self.filter_frame)
        self.keyword_entry.grid(row=0, column=1, padx=5, pady=5, sticky="ew")

        ttk.Label(self.filter_frame, text="Дата от:").grid(row=1, column=0, padx=5, pady=5)
        self.min_date_entry = DateEntry(self.filter_frame, date_pattern='y-mm-dd')
        self.min_date_entry.grid(row=1, column=1, padx=5, pady=5, sticky="ew")

        ttk.Label(self.filter_frame, text="Дата до:").grid(row=2, column=0, padx=5, pady=5)
        self.max_date_entry = DateEntry(self.filter_frame, date_pattern='y-mm-dd')
        self.max_date_entry.grid(row=2, column=1, padx=5, pady=5, sticky="ew")

        ttk.Label(self.filter_frame, text="Фильтр по комментариям:").grid(row=3, column=0, padx=5, pady=5)
        self.comment_filter_entry = ttk.Entry(self.filter_frame)
        self.comment_filter_entry.grid(row=3, column=1, padx=5, pady=5, sticky="ew")

        self.group_by_short_url_var = tk.BooleanVar()
        ttk.Checkbutton(
            self.filter_frame, 
            text="Группировать по короткому url", 
            variable=self.group_by_short_url_var,
            command=self.refresh_table
        ).grid(row=4, column=2, padx=5, pady=5)

        # Кнопки управления
        button_frame = ttk.Frame(self.filter_frame)
        button_frame.grid(row=4, column=0, columnspan=2, pady=10)

        ttk.Button(button_frame, text="Применить фильтры", command=self.filter_data).pack(side=tk.LEFT, padx=5)
        ttk.Button(button_frame, text="Добавить комментарий ко всем", command=self.add_comment_to_all_filtered).pack(side=tk.LEFT, padx=5)
        ttk.Button(button_frame, text="Сбросить фильтры", command=self.reset_filters).pack(side=tk.LEFT, padx=5)
        
        # Таблица и скроллбары
        self.table_container = ttk.Frame(main_container, padding="10")
        self.table_container.pack(fill=tk.BOTH, expand=True)

        self.xscrollbar = ttk.Scrollbar(self.table_container, orient="horizontal")
        self.xscrollbar.pack(side=tk.BOTTOM, fill=tk.X)

        self.yscrollbar = ttk.Scrollbar(self.table_container, orient="vertical")
        self.yscrollbar.pack(side=tk.RIGHT, fill=tk.Y)

        columns = list(df.columns) if not df.empty else []
        self.table = ttk.Treeview(
            self.table_container,
            columns=columns,
            show="headings",
            yscrollcommand=self.yscrollbar.set,
            xscrollcommand=self.xscrollbar.set
        )

        for col in columns:
            self.table.heading(col, text=col)
            self.table.column(col, width=120, stretch=False)

        style = ttk.Style()
        style.configure('Separator.Treeview', background='gray', rowheight=1)
        self.table.tag_configure('separator', background='gray', font=('Helvetica', 1))

        self.table.pack(side=tk.LEFT, fill=tk.BOTH, expand=True)
        self.yscrollbar.config(command=self.table.yview)
        self.xscrollbar.config(command=self.table.xview)

        self.table.bind('<Double-1>', self.on_double_click)
        self.table.bind('<Button-1>', self.on_link_click)

            # Контейнер для кнопок под таблицей
        bottom_button_frame = ttk.Frame(main_container)
        bottom_button_frame.pack(fill=tk.X, pady=(0, 10),)

        # Кнопка "Новое окно" под таблицей
        ttk.Button(
            bottom_button_frame, 
            text="Новое окно", 
            command=self.open_new_window
        ).pack(padx=5, anchor = 'center')
    
    def open_new_window(self):
        """Открывает новое окно с теми же данными"""
        new_window = tk.Toplevel(self.root)
        new_window.title("Фильтр резюме HH (Новое окно)")
        new_window.geometry("1000x700")
        ApplicationWindow(new_window)
    
    def load_initial_data(self):
        """Загружает начальные данные"""
        global df, comments
        load_data()
        if 'comment' not in df.columns:
            df['comment'] = ''
    
    def reset_filters(self):
        """Сбрасывает все фильтры"""
        self.keyword_entry.delete(0, tk.END)
        self.min_date_entry.set_date(None)
        self.max_date_entry.set_date(None)
        self.comment_filter_entry.delete(0, tk.END)
        self.group_by_short_url_var.set(False)
        self.load_initial_data()
        self.refresh_table()
    
    def refresh_table(self):
        """Обновляет таблицу в соответствии с текущими настройками"""
        if self.group_by_short_url_var.get():
            self.update_table_with_grouping()
        else:
            self.update_table_without_grouping()
    
    def update_table_with_grouping(self):
        """Обновляет таблицу с группировкой"""
        self.table.delete(*self.table.get_children())
        
        if df.empty:
            self.table.insert("", "end", values=["Нет данных для отображения"] * len(df.columns))
            return
            
        if 'short_url' not in df.columns:
            messagebox.showwarning("Предупреждение", "Нет данных для группировки (отсутствует колонка 'short_url')")
            self.group_by_short_url_var.set(False)
            self.update_table_without_grouping()
            return
        
        try:
            grouped_df = df.sort_values('short_url')
            prev_short_url = None
            
            for index, row in grouped_df.iterrows():
                item_id = f"I{index}"
                values = list(row)
                current_short_url = row['short_url']
                
                if prev_short_url is not None and current_short_url != prev_short_url:
                    separator_id = f"sep_{prev_short_url}_{current_short_url}"
                    self.table.insert("", "end", iid=separator_id, values=[""]*len(values), tags=('separator',))
                
                self.table.insert("", "end", iid=item_id, values=values)
                prev_short_url = current_short_url
        except Exception as e:
            messagebox.showerror("Ошибка", f"Не удалось выполнить группировку: {str(e)}")
            self.group_by_short_url_var.set(False)
            self.update_table_without_grouping()

    def update_table_without_grouping(self):
        """Обновляет таблицу без группировки"""
        self.table.delete(*self.table.get_children())
        
        if df.empty:
            self.table.insert("", "end", values=["Нет данных для отображения"] * len(df.columns))
            return
            
        for index, row in df.iterrows():
            item_id = f"I{index}"
            values = list(row)
            self.table.insert("", "end", iid=item_id, values=values)
    
    def filter_data(self):
        """Применяет фильтры к данным"""
        keyword = self.keyword_entry.get().strip().lower()
        min_date = self.min_date_entry.get_date() if self.min_date_entry.get() else None
        max_date = self.max_date_entry.get_date() if self.max_date_entry.get() else None
        comment_filter = self.comment_filter_entry.get().strip().lower()

        global df
        filtered_df = load_data()
        if not filtered_df:
            return
            
        if keyword:
            df = df[df['full_url'].str.lower().str.contains(keyword, na=False)]
        
        if min_date:
            df = df[df['visit_time'].dt.date >= min_date]
        if max_date:
            df = df[df['visit_time'].dt.date <= max_date]
        
        if comment_filter:
            df = df[df['comment'].str.lower().str.contains(comment_filter, na=False)]

        self.refresh_table()
    
    def add_comment_to_all_filtered(self):
        """Добавляет комментарий ко всем отфильтрованным записям"""
        if df.empty:
            messagebox.showwarning("Предупреждение", "Нет данных для добавления комментария")
            return
        
        comment = simpledialog.askstring("Добавить комментарий", "Введите комментарий для всех отфильтрованных записей:")
        if comment is None:
            return
        
        if not comment.strip():
            if not messagebox.askyesno("Подтверждение", "Вы хотите очистить комментарии для всех отфильтрованных записей?"):
                return
        
        row_ids = df.index.tolist()
        
        if save_comments_to_db(row_ids, comment):
            for row_id in row_ids:
                df.at[row_id, 'comment'] = comment
                item_id = f"I{row_id}"
                if comment:
                    comments[item_id] = comment
                elif item_id in comments:
                    del comments[item_id]
            
            data_change_manager.notify()
            messagebox.showinfo("Успех", f"Комментарий добавлен к {len(row_ids)} записям")
    
    def on_double_click(self, event):
        """Обработчик двойного клика для редактирования комментариев"""
        item = self.table.identify_row(event.y)
        column = self.table.identify_column(event.x)
        
        if item and column == '#{}'.format(len(df.columns) if df.empty else df.columns.get_loc('comment') + 1):
            current_value = comments.get(item, "")
            CommentEditor(self.root, item, current_value)
    
    def on_link_click(self, event):
        """Обработчик клика по ссылке"""
        item = self.table.identify_row(event.y)
        col = self.table.identify_column(event.x)
        
        if item and self.table.heading(col)['text'] == 'full_url':
            url = self.table.item(item, 'values')[df.columns.get_loc('full_url')]
            if url and isinstance(url, str) and url.startswith(('http://', 'https://')):
                webbrowser.open(url)

def main():
    """Основная функция запуска приложения"""
    root = tk.Tk()
    app = ApplicationWindow(root)
    root.mainloop()

if __name__ == "__main__":
    main()

C:\Users\sekin\AppData\Local\Temp\ipykernel_11960\2446055122.py:60: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, connect)


In [13]:
import tkinter as tk
from tkinter import ttk, messagebox, simpledialog
import pandas as pd
import psycopg2
import webbrowser
from typing import List, Callable
from datetime import datetime
from tkcalendar import DateEntry


class DataChangeManager:
    """Класс для управления уведомлениями об изменениях данных"""
    def __init__(self):
        self._subscribers: List[Callable] = []
    
    def subscribe(self, callback: Callable):
        self._subscribers.append(callback)
    
    def unsubscribe(self, callback: Callable):
        if callback in self._subscribers:
            self._subscribers.remove(callback)
    
    def notify(self):
        for callback in self._subscribers:
            callback()

data_change_manager = DataChangeManager()

DB_CONFIG = {
    'host': 'localhost',
    'port': '5433',
    'database': 'hh_test',
    'user': 'postgres',
    'password': '123'
}

df = pd.DataFrame()
comments = {}
comment_templates = []

def load_data():
    """Загружает данные из базы данных"""
    global df, comments, comment_templates
    try:
        with psycopg2.connect(**DB_CONFIG) as connect:
            with connect.cursor() as cursor:
                # Проверяем и создаем таблицу для комментариев, если ее нет
                cursor.execute("""
                    SELECT column_name 
                    FROM information_schema.columns 
                    WHERE table_name='info_res' AND column_name='comment'
                """)
                if not cursor.fetchone():
                    cursor.execute("ALTER TABLE info_res ADD COLUMN comment TEXT")
                    connect.commit()
                
                # Проверяем и создаем таблицу для шаблонов комментариев, если ее нет
                cursor.execute("""
                    SELECT EXISTS (
                        SELECT FROM information_schema.tables 
                        WHERE table_name = 'comment_templates'
                    )
                """)
                if not cursor.fetchone()[0]:
                    cursor.execute("""
                        CREATE TABLE comment_templates (
                            id SERIAL PRIMARY KEY,
                            template_text TEXT NOT NULL UNIQUE
                        )
                    """)
                    connect.commit()
            
            # Загружаем данные о резюме
            query = '''SELECT info_res.session_id, name_resume, full_url, img_text, 
                      short_url, visit_time, file_creation_time, type_file, comment 
                      FROM info_res
                      LEFT JOIN parsing_metadata ON info_res.session_id = parsing_metadata.session_id
                      ORDER BY info_res.id ASC'''
            
            df = pd.read_sql(query, connect)
            df['visit_time'] = pd.to_datetime(df['visit_time'])
            
            comments = {f"I{idx}": row['comment'] for idx, row in df.iterrows() 
                       if pd.notna(row['comment'])}
            
            # Загружаем шаблоны комментариев
            template_query = "SELECT template_text FROM comment_templates ORDER BY id"
            comment_templates = [row[0] for row in pd.read_sql(template_query, connect).values]
            
        return True
    except Exception as e:
        messagebox.showerror("Ошибка", f"Не удалось загрузить данные: {e}")
        return False

def save_comment_to_db(row_id, comment):
    """Сохраняет комментарий в базе данных"""
    try:
        with psycopg2.connect(**DB_CONFIG) as connect:
            with connect.cursor() as cursor:
                cursor.execute(
                    "UPDATE info_res SET comment = %s WHERE ctid = "
                    "(SELECT ctid FROM info_res ORDER BY id LIMIT 1 OFFSET %s)",
                    (comment, row_id))
                connect.commit()
        return True
    except Exception as e:
        messagebox.showerror("Ошибка", f"Не удалось сохранить комментарий: {e}")
        return False

def save_comments_to_db(row_ids, comment):
    """Сохраняет комментарии для нескольких записей"""
    try:
        with psycopg2.connect(**DB_CONFIG) as connect:
            with connect.cursor() as cursor:
                for row_id in row_ids:
                    cursor.execute(
                        "UPDATE info_res SET comment = %s WHERE ctid = "
                        "(SELECT ctid FROM info_res ORDER BY id LIMIT 1 OFFSET %s)",
                        (comment, row_id))
                connect.commit()
        return True
    except Exception as e:
        messagebox.showerror("Ошибка", f"Не удалось сохранить комментарии: {e}")
        return False

def add_comment_template(template_text):
    """Добавляет шаблон комментария в базу данных"""
    try:
        with psycopg2.connect(**DB_CONFIG) as connect:
            with connect.cursor() as cursor:
                # Проверяем, существует ли уже такой шаблон
                cursor.execute(
                    "SELECT EXISTS (SELECT 1 FROM comment_templates WHERE template_text = %s)",
                    (template_text,)
                )
                if cursor.fetchone()[0]:
                    messagebox.showwarning("Предупреждение", "Такой шаблон уже существует")
                    return False
                
                cursor.execute(
                    "INSERT INTO comment_templates (template_text) VALUES (%s)",
                    (template_text,)
                )
                connect.commit()
        return True
    except Exception as e:
        messagebox.showerror("Ошибка", f"Не удалось добавить шаблон: {e}")
        return False

def remove_comment_template(template_text):
    """Удаляет шаблон комментария из базы данных"""
    try:
        with psycopg2.connect(**DB_CONFIG) as connect:
            with connect.cursor() as cursor:
                cursor.execute(
                    "DELETE FROM comment_templates WHERE template_text = %s",
                    (template_text,))
                connect.commit()
        return True
    except Exception as e:
        messagebox.showerror("Ошибка", f"Не удалось удалить шаблон: {e}")
        return False

class CommentEditor:
    """Окно редактирования комментария"""
    def __init__(self, parent, item, current_value):
        self.editor = tk.Toplevel(parent)
        self.editor.title("Редактирование комментария")
        self.editor.geometry("500x400")
        self.editor.grab_set()
        
        self.parent = parent
        self.item = item
        self.row_id = int(item[1:])
        
        main_frame = ttk.Frame(self.editor)
        main_frame.pack(fill=tk.BOTH, expand=True, padx=10, pady=10)
        
        # Фрейм для шаблонов
        template_frame = ttk.LabelFrame(main_frame, text="Шаблоны комментариев", padding=5)
        template_frame.pack(fill=tk.X, pady=(0, 10))
        
        self.template_var = tk.StringVar()
        self.template_combobox = ttk.Combobox(
            template_frame, 
            textvariable=self.template_var,
            values=comment_templates,
            state="readonly"
        )
        self.template_combobox.pack(side=tk.LEFT, fill=tk.X, expand=True, padx=(0, 5))
        
        ttk.Button(
            template_frame, 
            text="Применить", 
            command=self.apply_template
        ).pack(side=tk.LEFT)
        
        ttk.Button(
            template_frame, 
            text="Удалить шаблон", 
            command=self.remove_template
        ).pack(side=tk.LEFT, padx=(5, 0))
        
        # Текстовое поле для комментария
        self.text_area = tk.Text(main_frame, wrap=tk.WORD, height=10)
        self.text_area.insert("1.0", current_value)
        self.text_area.pack(fill=tk.BOTH, expand=True)
        
        # Фрейм для кнопок
        button_frame = ttk.Frame(main_frame)
        button_frame.pack(fill=tk.X, pady=(10, 0))
        
        ttk.Button(button_frame, text="Сохранить", command=self.save_changes).pack(side=tk.LEFT, padx=5)
        ttk.Button(button_frame, text="Сохранить как шаблон", command=self.save_as_template).pack(side=tk.LEFT, padx=5)
        ttk.Button(button_frame, text="Очистить", command=self.clear_comment).pack(side=tk.LEFT, padx=5)
        ttk.Button(button_frame, text="Отмена", command=self.editor.destroy).pack(side=tk.RIGHT, padx=5)
        
        self.text_area.focus_set()
    
    def apply_template(self):
        """Применяет выбранный шаблон"""
        selected_template = self.template_var.get()
        if selected_template:
            self.text_area.delete("1.0", tk.END)
            self.text_area.insert("1.0", selected_template)
    
    def save_as_template(self):
        """Сохраняет текущий комментарий как шаблон"""
        template_text = self.text_area.get("1.0", "end-1c")
        if template_text:
            if add_comment_template(template_text):
                # Обновляем список шаблонов из базы данных
                global comment_templates
                with psycopg2.connect(**DB_CONFIG) as connect:
                    template_query = "SELECT template_text FROM comment_templates ORDER BY id"
                    comment_templates = [row[0] for row in pd.read_sql(template_query, connect).values]

                self.template_combobox['values'] = comment_templates
                self.template_var.set(template_text)
                messagebox.showinfo("Успех", "Шаблон успешно сохранен")
        else:
            messagebox.showwarning("Предупреждение", "Нельзя сохранить пустой шаблон")
    
    def remove_template(self):
        """Удаляет выбранный шаблон"""
        selected_template = self.template_var.get()
        if selected_template:
            if messagebox.askyesno("Подтверждение", f"Вы уверены, что хотите удалить шаблон '{selected_template}'?"):
                if remove_comment_template(selected_template):
                    comment_templates.remove(selected_template)
                    self.template_combobox['values'] = comment_templates
                    self.template_var.set('')
                    messagebox.showinfo("Успех", "Шаблон успешно удален")
                else:
                    messagebox.showerror("Ошибка", "Не удалось удалить шаблон")
    
    def save_changes(self):
        """Сохраняет изменения комментария"""
        new_comment = self.text_area.get("1.0", "end-1c")
        if save_comment_to_db(self.row_id, new_comment):
            df.at[self.row_id, 'comment'] = new_comment
            if new_comment:
                comments[self.item] = new_comment
            elif self.item in comments:
                del comments[self.item]
            data_change_manager.notify()
            self.editor.destroy()
    
    def clear_comment(self):
        """Очищает комментарий"""
        if save_comment_to_db(self.row_id, ""):
            df.at[self.row_id, 'comment'] = ""
            if self.item in comments:
                del comments[self.item]
            data_change_manager.notify()
            self.editor.destroy()

class ApplicationWindow:
    """Главное окно приложения"""
    def __init__(self, root):
        self.root = root
        self.root.title("Фильтр резюме HH с комментариями")
        self.root.geometry("1000x700")
        
        self.load_initial_data()
        self.setup_ui()
        data_change_manager.subscribe(self.on_data_changed)
        self.root.protocol("WM_DELETE_WINDOW", self.on_close)
        
        self.refresh_table()
    
    def on_close(self):
        """Обработчик закрытия окна"""
        data_change_manager.unsubscribe(self.on_data_changed)
        self.root.destroy()
    
    def on_data_changed(self):
        """Обработчик изменения данных"""
        self.load_initial_data()
        self.refresh_table()
    
    def setup_ui(self):
        """Настройка пользовательского интерфейса"""
        main_container = ttk.Frame(self.root)
        main_container.pack(fill=tk.BOTH, expand=True)

        self.filter_frame = ttk.Frame(main_container, padding="10")
        self.filter_frame.pack(fill=tk.X)

        # Элементы фильтрации
        ttk.Label(self.filter_frame, text="Ключевое слово в URL:").grid(row=0, column=0, padx=5, pady=5)
        self.keyword_entry = ttk.Entry(self.filter_frame)
        self.keyword_entry.grid(row=0, column=1, padx=5, pady=5, sticky="ew")

        ttk.Label(self.filter_frame, text="Дата от:").grid(row=1, column=0, padx=5, pady=5)
        self.min_date_entry = DateEntry(self.filter_frame, date_pattern='y-mm-dd')
        self.min_date_entry.grid(row=1, column=1, padx=5, pady=5, sticky="ew")

        ttk.Label(self.filter_frame, text="Дата до:").grid(row=2, column=0, padx=5, pady=5)
        self.max_date_entry = DateEntry(self.filter_frame, date_pattern='y-mm-dd')
        self.max_date_entry.grid(row=2, column=1, padx=5, pady=5, sticky="ew")

        # Фильтр по комментариям с выпадающим списком шаблонов
        ttk.Label(self.filter_frame, text="Фильтр по комментариям:").grid(row=3, column=0, padx=5, pady=5)
        
        comment_filter_frame = ttk.Frame(self.filter_frame)
        comment_filter_frame.grid(row=3, column=1, sticky="ew")
        
        self.comment_filter_var = tk.StringVar()
        self.comment_filter_combobox = ttk.Combobox(
            comment_filter_frame,
            textvariable=self.comment_filter_var,
            values=comment_templates,
            state="normal"
        )
        self.comment_filter_combobox.pack(side=tk.LEFT, fill=tk.X, expand=True, padx=(0, 5))
        
        ttk.Button(
            comment_filter_frame,
            text="Применить шаблон",
            command=self.apply_template_filter
        ).pack(side=tk.LEFT)

        self.group_by_short_url_var = tk.BooleanVar()
        ttk.Checkbutton(
            self.filter_frame, 
            text="Группировать по короткому url", 
            variable=self.group_by_short_url_var,
            command=self.refresh_table
        ).grid(row=4, column=2, padx=5, pady=5)

        # Кнопки управления
        button_frame = ttk.Frame(self.filter_frame)
        button_frame.grid(row=4, column=0, columnspan=2, pady=10)

        ttk.Button(button_frame, text="Применить фильтры", command=self.filter_data).pack(side=tk.LEFT, padx=5)
        ttk.Button(button_frame, text="Добавить комментарий ко всем", command=self.add_comment_to_all_filtered).pack(side=tk.LEFT, padx=5)
        ttk.Button(button_frame, text="Сбросить фильтры", command=self.reset_filters).pack(side=tk.LEFT, padx=5)
        
        # Таблица и скроллбары
        self.table_container = ttk.Frame(main_container, padding="10")
        self.table_container.pack(fill=tk.BOTH, expand=True)

        self.xscrollbar = ttk.Scrollbar(self.table_container, orient="horizontal")
        self.xscrollbar.pack(side=tk.BOTTOM, fill=tk.X)

        self.yscrollbar = ttk.Scrollbar(self.table_container, orient="vertical")
        self.yscrollbar.pack(side=tk.RIGHT, fill=tk.Y)

        columns = list(df.columns) if not df.empty else []
        self.table = ttk.Treeview(
            self.table_container,
            columns=columns,
            show="headings",
            yscrollcommand=self.yscrollbar.set,
            xscrollcommand=self.xscrollbar.set
        )

        for col in columns:
            self.table.heading(col, text=col)
            self.table.column(col, width=120, stretch=False)

        style = ttk.Style()
        style.configure('Separator.Treeview', background='gray', rowheight=1)
        self.table.tag_configure('separator', background='gray', font=('Helvetica', 1))

        self.table.pack(side=tk.LEFT, fill=tk.BOTH, expand=True)
        self.yscrollbar.config(command=self.table.yview)
        self.xscrollbar.config(command=self.table.xview)

        self.table.bind('<Double-1>', self.on_double_click)
        self.table.bind('<Button-1>', self.on_link_click)

        # Контейнер для кнопок под таблицей
        bottom_button_frame = ttk.Frame(main_container)
        bottom_button_frame.pack(fill=tk.X, pady=(0, 10),)

        # Кнопка "Новое окно" под таблицей
        ttk.Button(
            bottom_button_frame, 
            text="Новое окно", 
            command=self.open_new_window
        ).pack(padx=5, anchor='center')
    
    def apply_template_filter(self):
        """Применяет выбранный шаблон как фильтр"""
        selected_template = self.comment_filter_var.get()
        if selected_template:
            self.filter_data()
    
    def open_new_window(self):
        """Открывает новое окно с теми же данными"""
        new_window = tk.Toplevel(self.root)
        new_window.title("Фильтр резюме HH (Новое окно)")
        new_window.geometry("1000x700")
        ApplicationWindow(new_window)
    
    def load_initial_data(self):
        """Загружает начальные данные"""
        global df, comments, comment_templates
        load_data()
        if 'comment' not in df.columns:
            df['comment'] = ''
        
        # Обновляем выпадающий список шаблонов
        if hasattr(self, 'comment_filter_combobox'):
            self.comment_filter_combobox['values'] = comment_templates
    
    def reset_filters(self):
        """Сбрасывает все фильтры"""
        self.keyword_entry.delete(0, tk.END)
        self.min_date_entry.set_date(None)
        self.max_date_entry.set_date(None)
        self.comment_filter_var.set('')
        self.group_by_short_url_var.set(False)
        self.load_initial_data()
        self.refresh_table()
    
    def refresh_table(self):
        """Обновляет таблицу в соответствии с текущими настройками"""
        if self.group_by_short_url_var.get():
            self.update_table_with_grouping()
        else:
            self.update_table_without_grouping()
    
    def update_table_with_grouping(self):
        """Обновляет таблицу с группировкой"""
        self.table.delete(*self.table.get_children())
        
        if df.empty:
            self.table.insert("", "end", values=["Нет данных для отображения"] * len(df.columns))
            return
            
        if 'short_url' not in df.columns:
            messagebox.showwarning("Предупреждение", "Нет данных для группировки (отсутствует колонка 'short_url')")
            self.group_by_short_url_var.set(False)
            self.update_table_without_grouping()
            return
        
        try:
            grouped_df = df.sort_values('short_url')
            prev_short_url = None
            
            for index, row in grouped_df.iterrows():
                item_id = f"I{index}"
                values = list(row)
                current_short_url = row['short_url']
                
                if prev_short_url is not None and current_short_url != prev_short_url:
                    separator_id = f"sep_{prev_short_url}_{current_short_url}"
                    self.table.insert("", "end", iid=separator_id, values=[""]*len(values), tags=('separator',))
                
                self.table.insert("", "end", iid=item_id, values=values)
                prev_short_url = current_short_url
        except Exception as e:
            messagebox.showerror("Ошибка", f"Не удалось выполнить группировку: {str(e)}")
            self.group_by_short_url_var.set(False)
            self.update_table_without_grouping()

    def update_table_without_grouping(self):
        """Обновляет таблицу без группировки"""
        self.table.delete(*self.table.get_children())
        
        if df.empty:
            self.table.insert("", "end", values=["Нет данных для отображения"] * len(df.columns))
            return
            
        for index, row in df.iterrows():
            item_id = f"I{index}"
            values = list(row)
            self.table.insert("", "end", iid=item_id, values=values)
    
    def filter_data(self):
        """Применяет фильтры к данным"""
        keyword = self.keyword_entry.get().strip().lower()
        min_date = self.min_date_entry.get_date() if self.min_date_entry.get() else None
        max_date = self.max_date_entry.get_date() if self.max_date_entry.get() else None
        comment_filter = self.comment_filter_var.get().strip().lower()

        global df
        filtered_df = load_data()
        if not filtered_df:
            return
            
        if keyword:
            df = df[df['full_url'].str.lower().str.contains(keyword, na=False)]
        
        if min_date:
            df = df[df['visit_time'].dt.date >= min_date]
        if max_date:
            df = df[df['visit_time'].dt.date <= max_date]
        
        if comment_filter:
            df = df[df['comment'].str.lower().str.contains(comment_filter, na=False)]

        self.refresh_table()
    
    def add_comment_to_all_filtered(self):
        """Добавляет комментарий ко всем отфильтрованным записям"""
        if df.empty:
            messagebox.showwarning("Предупреждение", "Нет данных для добавления комментария")
            return
        
        # Создаем окно для выбора шаблона или ввода нового комментария
        dialog = tk.Toplevel(self.root)
        dialog.title("Добавить комментарий ко всем")
        dialog.geometry("500x300")
        dialog.grab_set()
        
        main_frame = ttk.Frame(dialog, padding=10)
        main_frame.pack(fill=tk.BOTH, expand=True)
        
        # Выбор шаблона
        ttk.Label(main_frame, text="Выберите шаблон:").pack(anchor=tk.W)
        
        template_var = tk.StringVar()
        template_combobox = ttk.Combobox(
            main_frame,
            textvariable=template_var,
            values=comment_templates,
            state="readonly"
        )
        template_combobox.pack(fill=tk.X, pady=(0, 10))
        
        # Или ввод нового комментария
        ttk.Label(main_frame, text="Или введите новый комментарий:").pack(anchor=tk.W)
        
        text_area = tk.Text(main_frame, wrap=tk.WORD, height=5)
        text_area.pack(fill=tk.BOTH, expand=True)
        
        # Кнопки
        button_frame = ttk.Frame(main_frame)
        button_frame.pack(fill=tk.X, pady=(10, 0))
        
        def apply_comment():
            comment = template_var.get() if template_var.get() else text_area.get("1.0", "end-1c")
            if not comment:
                if not messagebox.askyesno("Подтверждение", "Вы хотите очистить комментарии для всех отфильтрованных записей?"):
                    return
            
            row_ids = df.index.tolist()
            
            if save_comments_to_db(row_ids, comment):
                for row_id in row_ids:
                    df.at[row_id, 'comment'] = comment
                    item_id = f"I{row_id}"
                    if comment:
                        comments[item_id] = comment
                    elif item_id in comments:
                        del comments[item_id]
                
                data_change_manager.notify()
                dialog.destroy()
                messagebox.showinfo("Успех", f"Комментарий добавлен к {len(row_ids)} записям")
        
        ttk.Button(button_frame, text="Применить", command=apply_comment).pack(side=tk.LEFT, padx=5)
        ttk.Button(button_frame, text="Отмена", command=dialog.destroy).pack(side=tk.RIGHT, padx=5)
    
    def on_double_click(self, event):
        """Обработчик двойного клика для редактирования комментариев"""
        item = self.table.identify_row(event.y)
        column = self.table.identify_column(event.x)
        
        if item and column == '#{}'.format(len(df.columns) if df.empty else df.columns.get_loc('comment') + 1):
            current_value = comments.get(item, "")
            CommentEditor(self.root, item, current_value)
    
    def on_link_click(self, event):
        """Обработчик клика по ссылке"""
        item = self.table.identify_row(event.y)
        col = self.table.identify_column(event.x)
        
        if item and self.table.heading(col)['text'] == 'full_url':
            url = self.table.item(item, 'values')[df.columns.get_loc('full_url')]
            if url and isinstance(url, str) and url.startswith(('http://', 'https://')):
                webbrowser.open(url)

def main():
    """Основная функция запуска приложения"""
    root = tk.Tk()
    app = ApplicationWindow(root)
    root.mainloop()

if __name__ == "__main__":
    main() 

C:\Users\sekin\AppData\Local\Temp\ipykernel_11960\2828645219.py:80: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, connect)
C:\Users\sekin\AppData\Local\Temp\ipykernel_11960\2828645219.py:88: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  comment_templates = [row[0] for row in pd.read_sql(template_query, connect).values]
